# DAY4 AOC

In [4]:
# import libraries
import pandas as pd

## PART 1
"Looks like the Chief's not here. Next!" One of The Historians pulls out a device and pushes the only button on it. After a brief flash, you recognize the interior of the Ceres monitoring station!

As the search for the Chief continues, a small Elf who lives on the station tugs on your shirt; she'd like to know if you could help her with her word search (your puzzle input). She only has to find one word: XMAS.

This word search allows words to be horizontal, vertical, diagonal, written backwards, or even overlapping other words. It's a little unusual, though, as you don't merely need to find one instance of XMAS - you need to find all of them. Here are a few ways XMAS might appear, where irrelevant characters have been replaced with .:`

### STRATEGY 

assumes letters surrounding letter of interest (LETTER) has indices as follows: 

[[1,2,3],

[4,LETTER,5],

 [6,7,8]] 

so for example, letters at the top left corner, will have letters surrounding it with indices [5,7,8]
so for example, letters at the bottom right corner, will have letters surrounding it with indices [1,2,4]

In [49]:
# function to get indices surrounding letter
def extract_index_surrounding_letters(row,col,total_row,total_col):
    #since the index starts at 0, not 1
    total_row = total_row - 1 
    total_col = total_col - 1 

    # this is for top row 
    if(row < 1):
        if(col <1): # this is for the first column of the first row
            index_pos = [5,7,8]
        elif(col == total_col): # this is for the last column of the first row
            index_pos = [4,6,7]
        else: # this is for the other columsn of the first row
            index_pos = [4,5,6,7,8]

    # this is for the bottom row
    elif(row == total_row):
        if(col<1):# this is for the first column of the bottom row
            index_pos = [2,3,5]
        elif(col == total_col): #this is for the last column of the last row 
            index_pos = [1,2,4]
        else: # this is for the other columns of the last row
            index_pos = [1,2,3,4,5]
            
    # this is for the rows in between
    else: 
        if(col < 1): #this is for the first column of the in between rows
            index_pos = [2,3,5,7,8]
        elif(col == total_col): #this is for the last column of the in between rows
            index_pos = [1,2,4,6,7]
        else: #this is for the other columns of the in between rows
            index_pos = [1,2,3,4,5,6,7,8]

    return(index_pos)


# extract new row and col based on index_val (from function above) and starting col and row
def new_col_row_based_on_index(index_val,start_row,start_col):
    new_row_col = pd.DataFrame()

    #top rows
    if(index_val in [1,2,3]):
        new_row = start_row - 1
        if(index_val == 1):
            new_col = start_col - 1
        elif(index_val ==2):
            new_col = start_col
        else:
            new_col = start_col + 1
    #middle rows
    elif(index_val in [4,5]):
        new_row = start_row
        if(index_val == 4):
            new_col = start_col - 1
        else:
            new_col = start_col + 1
    #bottom rows
    else:
        new_row = start_row + 1
        if(index_val ==6):
            new_col = start_col - 1
        elif(index_val == 7):
            new_col = start_col
        else:
            new_col = start_col + 1

    new_row_col = [new_row, new_col]

    return(new_row_col)
    


            

In [272]:
# function to read file
def read_text_file_to_dataframe(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Remove whitespace and create a list of strings
    data = [list(line) for line in lines]

    # Create a dataframe
    df = pd.DataFrame(data)

    return df

In [345]:
data = read_text_file_to_dataframe('input/day4.txt')

In [280]:
total_row = data.shape[0]
total_col = data.shape[1]

In [308]:
total_Xmas = 0
for temp_row in range(0,(total_row)):
    # checking to see if and WHERE X is located in every row
    check_row = data.iloc[[temp_row]]
    X_col_pos = check_row.iloc[0][check_row.iloc[0] == 'X'].index.tolist()
    if (X_col_pos != '[]'): # if there is X, moving on
        for temp_col in X_col_pos: #this iterates through all the "X" in the respective row

            #getting all the indices of the surrounding X
            temp_index_surrounding_X = (extract_index_surrounding_letters(temp_row,temp_col,total_row,total_col))

            #getting the letters of the surrounding X
            surrounding_X = []
            for temp_index in temp_index_surrounding_X:
                temp_letter = new_col_row_based_on_index(temp_index,temp_row,temp_col)
                surrounding_X.append(data.iloc[temp_letter[0],temp_letter[1]])

            # checking to see if 'M' is around X
            if 'M' in surrounding_X:
                location_M_in_vect = [i for i, value in enumerate(surrounding_X) if value == 'M']
                # iterating through all the surrounding M's
                for temp_M_loc in location_M_in_vect: 
                    index_of_M = temp_index_surrounding_X[temp_M_loc]
                    M_col_row_df = new_col_row_based_on_index(index_of_M, temp_row,temp_col)
                    M_row = M_col_row_df[0]
                    M_col = M_col_row_df[1]

                    #getting the indices of all the surrouding letters of M
                    acceptable_A_indices = extract_index_surrounding_letters(M_row,M_col,total_row,total_col)
                    
                    # since the word 'XMAS' must be in same direction, 'A' must have the same index position as 'M' relative to 'X'
                    # this also accounts for 'edge' cases, if 'XM' stops at the end of the string
                    if(index_of_M in acceptable_A_indices):
                        A_col_row_df = new_col_row_based_on_index(index_of_M,M_row,M_col)
                        A_row = A_col_row_df[0]
                        A_col = A_col_row_df[1]
                        test_A_letter = data.iloc[A_row,A_col]
                        # testing if A is in the respective index
                        if(test_A_letter == "A"):

                            acceptable_S_indices = extract_index_surrounding_letters(A_row,A_col,total_row,total_col)
                            
                            # since the word 'XMAS' must be in same direction, 'S' must have the same index position as 'A' and 'M' relative to 'X'
                            if(index_of_M in acceptable_S_indices):
                                S_col_row_df = new_col_row_based_on_index(index_of_M,A_row,A_col)
                                
                                S_row = S_col_row_df[0]
                                S_col = S_col_row_df[1]
                            
                                test_S_letter = data.iloc[S_row,S_col]
                                if(test_S_letter == 'S' ):
                                    # FINAL COUNT
                                    total_Xmas = total_Xmas + 1
                                else:
                                    next
                        else:
                            next
            else:
                next

print(total_Xmas)


2603


## PART 2

Looking for the instructions, you flip over the word search to find that this isn't actually an XMAS puzzle; it's an X-MAS puzzle in which you're supposed to find two MAS in the shape of an X. One way to achieve that is like this:

M.S
.A.
M.S

### Strategy:
Check that only two letters are in the indices of interest [1,3,5,8] and that there must be diffferent letters in the opposite corners [aka 1,5 and 3,8] 

In [346]:
total_Xmas = 0
must_have_indices = [1,3,6,8] 
must_be_letters = ['M','S']
part2_sum = 0

total_row = data.shape[0]
total_col = data.shape[1]
for temp_row in range(0,(total_row)):
    check_row = data.iloc[[temp_row]]
    #checks to see where or IF A's are in the row
    A_col_pos = check_row.iloc[0][check_row.iloc[0] == 'A'].index.tolist()
    if (A_col_pos != '[]'):
        #this iterates through all the "A" in the respective row
        for temp_col in A_col_pos: 
            temp_index_surrounding_A = (extract_index_surrounding_letters(temp_row,temp_col,total_row,total_col))
            # checking to see if the must have indices (top left, top right, bottom left, bottom right' AKA [1,3,6,8] are there
            if(all(item in temp_index_surrounding_A for item in must_have_indices)):
                test_letters_A = []
                # iterating through all the 
                for temp_index in must_have_indices :
                    temp_letter = new_col_row_based_on_index(temp_index,temp_row,temp_col)
                    test_letters_A.append(data.iloc[temp_letter[0],temp_letter[1]])
                    #checking that 'M' and 'S' are the only letters in the corners surrounding A
                if(all(item in must_be_letters for item in test_letters_A)):
                    #making sure 'M' and 'S' are in opposite corners
                    if(test_letters_A[0]!=test_letters_A[3] and test_letters_A[1]!=test_letters_A[2]):
                        # answer
                        part2_sum = part2_sum + 1
                        

                    


In [347]:
part2_sum

1965